<a href="https://colab.research.google.com/github/Yunpei24/BigDataBase/blob/main/TP4_Programmation_avanc%C3%A9e_avec_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Le but principal de ce notebook est de maîtriser <b>Spark SQL</b> et de pouvoir défnir des fonctions de fenêtre Spark et des fonctions Python enregistrées dans le registre Spark afin d'être utilisées dans les requêtes SQL.

Le même jeu de données du précédent TP sera utilisé avec le même environnement d'exécution.


# Initialisation de l'environnement d'exécution

Installation du JDK

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Téléchargement de l'archive du framework Apache Spark

In [2]:
# Download Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

Extraction de l'archive dans le dossier courant <mark>/content</mark>

In [3]:
# Unzip the file
!tar xf spark-3.3.1-bin-hadoop3.tgz

Installation des modules Python <b>pyspark</b> et <b>findspark</b>

In [4]:
!pip install -q pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.1 MB/s eta 0:00:00


Test de l'installation de pyspark

In [5]:
!find /content -name "pyspark"

/content/spark-3.3.1-bin-hadoop3/python/pyspark
/content/spark-3.3.1-bin-hadoop3/python/pyspark/python/pyspark
/content/spark-3.3.1-bin-hadoop3/bin/pyspark


Création des variables d'environnement <mark>SPARK_HOME</mark> et <mark>JAVA_HOME</mark> pour situer respectivement les emplacements d'installation de Spark et Java 

In [6]:
import os
os.environ["SPARK_HOME"] =  "/content/spark-3.3.1-bin-hadoop3" 
os.environ["JAVA_HOME"] ="/usr/lib/jvm/java-8-openjdk-amd64"

Importation des bibliothèques Spark SQL

In [7]:
import findspark 
print("findspark.init() initialise les variables d'environnement pour spark") 
findspark.init() 

# Pyspark session objects
from pyspark.sql import SparkSession 
# Pyspark session configuration
from pyspark import SparkConf  

# Pyspark functions
import pyspark.sql.functions as f
from pyspark.sql import * 

# Pyspark SQL data types
from pyspark.sql.types import *

findspark.init() initialise les variables d'environnement pour spark


# Définition de fonctions utiles pour la suite

La fonction <mark>demarrer_spark</mark> permet d'initialiser une session <i>client</i> avec Spark

In [8]:
def demarrer_spark():
  local = "local[*]"
  appName = "TP3"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "100G").\
  set("spark.driver.memory","50G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.driver.maxResultSize", "10G")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  # spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")
  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  # spark.conf.set("spark.sql.shuffle.partitions","200")    

  print("session démarrée, son id est ", sc.applicationId)
  return spark

Démarrage de la session

In [9]:
spark = demarrer_spark()

session démarrée, son id est  local-1674566082656


En vue de simplifier l'exécution des requêtes SQL, nous définissons la commande magique &#128526; <b><font color="blue">%%sql</font></b> pour exécuter les requêtes plus facilement

In [10]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    resultat = None
    est_une_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          resultat = spark.sql(r)
          est_une_requete = r.lower().startswith('select') or r.lower().startswith('with')  
    if(est_une_requete):
      resultat.explain()
      return display(resultat)
    else:
      return print('ok')

De même, nous redéfinissons la fonction <b>display</b> pour un meilleur affichage des données manipulées.

In [11]:
import pandas as pd

def display(df, n=10):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()

print("display redéfini")

display redéfini


# Programmation avancée avec <i>Spark SQL</i>
Le travail consiste à utiliser des fonctions Python et <i>Spark SQL</i> avec le langage framework <i>pyspark</i> et d'effectuer des traitements distribués via des requêtes SQL. 

Vous devrez concevoir et écrire vous-même les requêtes à partir de la deuxième activité.

## Activité 1 : utilisation de la clause <i>OVER</i> avec Spark SQL

Téléchargement du jeu de données de notre dernier TP, en l'occurence le fichier <u>purchases.txt</u>

In [12]:
!curl -L -o 'purchases.txt' 'https://drive.google.com/u/0/uc?id=1NS-PSXW8bSNpzFH4XRbtmMnMGhXBdYy6&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  201M  100  201M    0     0   124M      0  0:00:01  0:00:01 --:--:--  162M


Affichage des premiers lignes du fichier. Le format des enregistrement est le suivant:
<table border='1'><tr>
<td>Date</td><td>Heure</td><td>Magasin</td><td>Produit</td><td>Montant</td><td>Moyen_de_paiement</td>
</tr></table>
La tabulation <b>\t</b> est utilisée comme séparateur de colonne ✅

In [13]:
!head -10  ./purchases.txt

2012-01-01	09:00	San Jose	Men's Clothing	214.05	Amex
2012-01-01	09:00	Fort Worth	Women's Clothing	153.57	Visa
2012-01-01	09:00	San Diego	Music	66.08	Cash
2012-01-01	09:00	Pittsburgh	Pet Supplies	493.51	Discover
2012-01-01	09:00	Omaha	Children's Clothing	235.63	MasterCard
2012-01-01	09:00	Stockton	Men's Clothing	247.18	MasterCard
2012-01-01	09:00	Austin	Cameras	379.6	Visa
2012-01-01	09:00	New York	Consumer Electronics	296.8	Cash
2012-01-01	09:00	Corpus Christi	Toys	25.38	Discover
2012-01-01	09:00	Fort Worth	Toys	213.88	Visa


Création du dataframe Spark <mark>df</mark> à partir du jeu de données du fichier <u>purchases.txt</u> téléchargé

In [14]:
schemaTable = StructType([
    StructField("Date", DateType(), True),
    StructField("Heure", StringType(), True),
    StructField("Magasin", StringType(), True),
    StructField("Produit", StringType(), True),
    StructField("Montant", DoubleType(), True),
    StructField("ModePaiement", StringType(), True)
    ])

df = spark.read.load("purchases.txt", format="csv", sep="\t", schema=schemaTable, header=False)

Affichage du dataframe

In [15]:
display(df)

,Date,Heure,Magasin,Produit,Montant,ModePaiement
0,2012-01-01,09:00,San Jose,Men's Clothing,214.05,Amex
1,2012-01-01,09:00,Fort Worth,Women's Clothing,153.57,Visa
2,2012-01-01,09:00,San Diego,Music,66.08,Cash
3,2012-01-01,09:00,Pittsburgh,Pet Supplies,493.51,Discover
4,2012-01-01,09:00,Omaha,Children's Clothing,235.63,MasterCard
5,2012-01-01,09:00,Stockton,Men's Clothing,247.18,MasterCard
6,2012-01-01,09:00,Austin,Cameras,379.60,Visa
7,2012-01-01,09:00,New York,Consumer Electronics,296.80,Cash
8,2012-01-01,09:00,Corpus Christi,Toys,25.38,Discover
9,2012-01-01,09:00,Fort Worth,Toys,213.88,Visa


Matérialisation du dataframe comme une vue SQL avec la vue <mark>purchases</mark> qui pointe sur lui.

In [16]:
df.createOrReplaceTempView('purchases')

Test de notre première requête SQL. <b>Que fait-elle ?</b>

In [ ]:
%%sql
WITH meilleurProduitsParMagasin AS (SELECT magasin, produit, RANK() OVER(PARTITION BY magasin ORDER BY montant DESC) AS classement
FROM purchases) SELECT magasin, produit FROM meilleurProduitsParMagasin WHERE classement <= 3 ORDER BY magasin ASC, classement DESC

<b>Que fait cette deuxième requête ?</b>
N'hésitez pas ! Exécutez la &#128521;

In [ ]:
%%sql
WITH cc AS (SELECT magasin, SUM(montant) AS total FROM purchases GROUP BY magasin), cc2 AS (SELECT magasin, total, ROW_NUMBER() over (ORDER BY total DESC) AS numero FROM cc) SELECT * FROM cc2 WHERE numero <= 5

In [ ]:
%%sql
WITH mp AS (select magasin, COUNT(Produit)) AS totalprod from purchases group by Produit), mp2 AS (SELECT magasin, totalprod, ROW_NUMBER() over (ORDER BY totalprod DESC) AS numero FROM mp) SELECT * FROM mp2 WHERE numero <= 3

## Activité 2
Nous continuons à travailler avec la même vue <mark>purchases</mark>. <u>Le but est d’écrire vos propres requêtes</u>.
<ol>
<li>Donner le top 3 des magasins pour chaque mode de paiement.</li>
<li>Donner le top 3 des magasins pour chaque produit.</li>
</ol>

In [ ]:
# 1. Donner le top 3 des magasins pour chaque mode de paiement.
%%sql 
WITH meilleurMagasinParModePaiement AS (SELECT magasin, ModePaiement, RANK() OVER(PARTITION BY magasin ORDER BY ModePaiement DESC) AS classement
FROM purchases) SELECT magasin, ModePaiement FROM meilleurMagasinParModePaiement WHERE classement <= 3 ORDER BY magasin ASC, classement DESC

In [ ]:
# 2. Donner le top 3 des magasins pour chaque produit.
%%sql
WITH meilleurMagasinParProduit AS (SELECT magasin, Produit, RANK() OVER(PARTITION BY magasin ORDER BY Produit DESC) AS classement
FROM purchases) SELECT magasin, Produit FROM meilleurMagasinParProduit WHERE classement <= 3 ORDER BY magasin ASC, classement DESC

## Activité 3 : utilisation de fonctions Python sous Spark SQL

Que fait la fonction suivante ?

In [ ]:
dayNames = ['', 'Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
  
def weekDayName(dayID):
  global dayNames
  val = int(dayID)
  if(1 <= val <= 7):
    return dayNames[val]
  else:
    return "Unknown"

Enregistrement de la fonction dans le registre des fonctionnalités de Spark SQL

In [ ]:
spark.udf.register("weekDayName", weekDayName, StringType())

Que fait la requête SQL ci-dessous ?

In [ ]:
%%sql
SELECT weekDayName(date_format(date, 'F')), SUM(montant) AS MontantTotal FROM purchases GROUP BY date_format(date, 'F')

## Activité 4
Créer vos propres fonctions et requêtes SQL pour répondre aux questions suivantes.
<ol>
<li>Quel est le chiffre d'affaire réalisé par chaque magasin en francs CFA ? <b>Utiliser une fonction Python pour la conversion</b> <mark>Le taux de change n'est pas fixe. Il évolue dans le temps selon le contexte socio-économique.</mark> <u>Exemple:</u> <a href="https://www.google.com/search?client=firefox-b-d&q=dollar+to+cfa">https://www.google.com/search?client=firefox-b-d&q=dollar+to+cfa</a></li>
</ol>

<mark>Pour convertir des devises, vous pouvez utiliser le module Python <a href="https://pypi.org/project/CurrencyConverter/">Currency Converter</a>.</mark> 
Dans les deux cellules qui suivent vous avez son installation et un exemple de conversion de 100 Euro en Dollar.

In [ ]:
!pip install currencyconverter

In [ ]:
from currency_converter import CurrencyConverter
converter = CurrencyConverter()
converter.convert(100, 'EUR', 'USD')

In [ ]:
# 1. Quel est le chiffre d'affaire réalisé par chaque magasin en francs CFA ?
%%sql


# Références
**Window Functions** : 
https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html

**Supported Data Types with Spark SQL** : https://spark.apache.org/docs/latest/sql-ref-datatypes.html

**Traitement de données massives avec Apache Spark** : http://b3d.bdpedia.fr/spark-batch.html

**PySpark : Tout savoir sur la librairie Python** : https://datascientest.com/pyspark